In [76]:
import pandas as pd
import numpy as np
from datetime import datetime as DateTime, timedelta as TimeDelta

In [80]:
docu = pd.ExcelFile('Covid Chile V2 PARA PRUEBAS.xlsx')

docuConfirma = pd.read_excel(docu,  sheet_name="COVID_CL_CONFIRMA")
docuMuertes = pd.read_excel(docu,  sheet_name="COVID_CL_MUERTE")
docuRecuperados = pd.read_excel(docu,  sheet_name="COVID_CL_RECUPERA")

#eliminación de encabezados Unnamed y filas NaN
docuConfirma.dropna(axis = 0, how = 'all', inplace = True)
if(docuConfirma.columns[0] == 'Unnamed: 0'):
    docuConfirma.columns = docuConfirma.iloc[0]
    docuConfirma = docuConfirma.iloc[1:,].reindex()
    
docuMuertes.dropna(axis = 0, how = 'all', inplace = True)
if(docuMuertes.columns[0] == 'Unnamed: 0'):
    docuMuertes.columns = docuMuertes.iloc[0]
    docuMuertes = docuMuertes.iloc[1:,].reindex()
    
docuRecuperados.dropna(axis = 0, how = 'all', inplace = True)
if(docuRecuperados.columns[0] == 'Unnamed: 0'):
    docuRecuperados.columns = docuRecuperados.iloc[0]
    docuRecuperados = docuRecuperados.iloc[1:,].reindex()
    
#creacion de lista fechas entre fecha minima y fecha actual
fechaMinima = docuConfirma.Fecha.min()
fechaMaxima = docuConfirma.Fecha.max()
fechaActual = DateTime.now()
fechaActual = fechaActual.replace(hour=0, minute=0, second=0, microsecond=0)
if(fechaActual < fechaMaxima):
    fechaActual = fechaMaxima
fechaFinal = fechaActual + TimeDelta(days=8)
listaFechas = []
listaFechas = pd.date_range(start=fechaMinima,end=fechaFinal)

#dataframe con columna Region, Comuna y Codigo comuna
df3 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"], "Codigo comuna" : docuConfirma["ID_Comuna"]})
res2 = df3.groupby(["Region","Comuna", "Codigo comuna"]).sum().reset_index()

#multiplicar lista de fechas por cantidad de comunas
lista = []
for Comuna in res2["Comuna"]:
    for Fecha in listaFechas:
        lista.append(Fecha)

#multiplicar cada fila por cantidad de fecha dentro del rango minimo y actual
res3 = res2.loc[np.repeat(res2.index.values, len(listaFechas))].reset_index(drop = True)
#implementacion de fecha en dataframe
res3['Fecha'] = lista
res3['Casos Diarios'] = 0
res3['Fallecidos Diarios'] = 0
res3['Recuperados Diarios'] = 0

#Columna Casos diarios en dataframe
df4 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"],"Fecha" : docuConfirma["Fecha"],"Codigo comuna" : docuConfirma["ID_Comuna"] ,"Casos Diarios" : 0})
resConfirma4 = df4.groupby(["Region","Comuna","Fecha",'Codigo comuna']).count().reset_index()
resConfirma4.insert(4, "Fallecidos Diarios", 0, True)
resConfirma4.insert(5, "Recuperados Diarios", 0, True)
resConfirma5 = pd.concat([ res3, resConfirma4 ])
resConfirma6 = resConfirma5.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

#Columna Fallecidos diarios en dataframe
df5 = pd.DataFrame({"Region" : docuMuertes["Region"], "Comuna" : docuMuertes["Comuna"], "Codigo comuna" : docuMuertes["ID_Comuna"] ,"Fecha" : docuMuertes["Fecha"], "Fallecidos Diarios" : 0})
resMuerte4 = df5.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resMuerte4.insert(4, "Casos Diarios", 0, True)
resMuerte4.insert(6, "Recuperados Diarios", 0, True) 
resMuerte5 = pd.concat([ resConfirma6 ,  resMuerte4])
resMuerte6 = resMuerte5.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

#Columna Recuperados diarios en dataframe
df6 = pd.DataFrame({"Region" : docuRecuperados["Region"], "Comuna" : docuRecuperados["Comuna"], "Codigo comuna" : docuRecuperados["ID_Comuna"],"Fecha" : docuRecuperados["Fecha"], "Recuperados Diarios" : 0})
resRecupera4 = df6.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resRecupera4.insert(4, "Casos Diarios", 0, True)
resRecupera4.insert(5, "Fallecidos Diarios", 0, True) 
resTotal = pd.concat([ resMuerte6 ,  resRecupera4])
resTotal = resTotal.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

#columnas Casos acumulados, Muertes, Recuperados, Casos Activos, Dia y on-off
resTotal.insert(5, "Casos Acumulados", resTotal['Casos Diarios'], True)
resTotal.insert(6, "Muertes", resTotal['Fallecidos Diarios'], True)
resTotal.insert(7, "Recuperados", resTotal['Recuperados Diarios'], True) 
resTotal.insert(8, "Casos Activos", resTotal['Casos Acumulados'], True)
resTotal['Dia'] = 0
resTotal['on-off'] = 'OFF'
inicial = True

for i in range(1, len(resTotal)):
    if(resTotal.loc[i, 'Comuna'] == resTotal.loc[i-1, 'Comuna']):
        resTotal.loc[i, 'Casos Acumulados'] = resTotal.loc[i, 'Casos Acumulados'] + resTotal.loc[i-1, 'Casos Acumulados']
        resTotal.loc[i, 'Muertes'] = resTotal.loc[i, 'Muertes'] + resTotal.loc[i-1, 'Muertes']
        resTotal.loc[i, 'Recuperados'] = resTotal.loc[i, 'Recuperados'] + resTotal.loc[i-1, 'Recuperados']
        resTotal.loc[i, 'Casos Activos'] = (resTotal.loc[i, 'Casos Acumulados'] - resTotal.loc[i, 'Muertes']) - resTotal.loc[i, 'Recuperados']
        if(resTotal.loc[i, 'Casos Activos'] < 0):
             resTotal.loc[i, 'Casos Activos'] = 0
        if(resTotal.loc[0, 'Casos Diarios'] > 0):
            resTotal.loc[0, 'Dia'] = 1
        if(resTotal.loc[i, 'Casos Diarios'] > 0):
            inicial = False
            resTotal.loc[i, 'on-off'] = 'START'
        if(inicial == False):
            resTotal.loc[i, 'Dia'] = 1 + resTotal.loc[i-1, 'Dia']
            if(resTotal.loc[i-1, 'on-off'] == 'OFF'):
                resTotal.loc[i, 'on-off'] = 'START'
            else: 
                if(resTotal.loc[i-1, 'on-off'] == 'START' or resTotal.loc[i-1, 'on-off'] == 'ON'):
                    resTotal.loc[i, 'on-off'] = 'ON'
            if(resTotal.loc[i, 'Fecha'] > fechaActual):
                resTotal.loc[i, 'on-off'] = 'OUT TIME' 
    else:
        inicial = True
        if(resTotal.loc[i, 'Casos Diarios'] > 0):
            resTotal.loc[i, 'Dia'] = 1
            resTotal.loc[i, 'on-off'] = 'START'
            inicial = False
            
#calcular columna Max Dia
resTotal['Max Dia'] = 0
maxDia = resTotal[resTotal.groupby(['Region', 'Comuna'])['Casos Diarios'].cumsum() != 0].reset_index(drop = True)
maxDia2 = maxDia[maxDia.Fecha < fechaActual].groupby(['Region', 'Comuna'])['Casos Diarios'].agg([('start','first'),('end','last'),('Max Dia','size')]).reset_index()
maxDia2['Max Dia'] = maxDia2['Max Dia']+1

#agregar valores a columna Max Dia
c = 0
for i in range(0, len(resTotal)):
    for j in range(c, len(maxDia2)):
        if(resTotal.loc[i, 'Comuna'] == maxDia2.loc[j, 'Comuna']):
            resTotal.loc[i, 'Max Dia'] = maxDia2.loc[j , 'Max Dia']
            c = j
            break
#ordenar columnas            
nuevo_orden = [0,1,3,2,4,5,6,8,7,10,11,9,12,13]
resTotal = resTotal[resTotal.columns[nuevo_orden]]
resTotal.to_csv("FINAL.csv", index=False)
